# Regenerative Potential Index — Master Pipeline

## Table of Contents
1. Setup  
2. Original Code  
3. Amsterdam District Processing (from manifest)  
4. Map Visualisation (Amsterdam districts)  
5. Histogram Generation per District  
6. Radar Charts per District  
7. Correlation Matrix of Indicators  
8. Scenario Engine (one-step)  
9. Time-Series Scenario Engine  
10. Story-Based Scenarios  
11. Plotly Dashboard Export (static for GitHub Pages)  
12. GitHub Actions Builder (workflow + build script)  


# 1. Setup

This section prepares the environment, imports packages and ensures the output folders exist.
It is designed to work both locally and in Google Colab.

In [1]:

import os

# Base output folders
OUTPUT_DIR = "RCI_outputs"
MAPS_DIR = os.path.join(OUTPUT_DIR, "maps")
HIST_DIR = os.path.join(OUTPUT_DIR, "histograms")
RADAR_DIR = os.path.join(OUTPUT_DIR, "radar")
SCENARIO_DIR = os.path.join(OUTPUT_DIR, "scenarios")

for d in [OUTPUT_DIR, MAPS_DIR, HIST_DIR, RADAR_DIR, SCENARIO_DIR]:
    os.makedirs(d, exist_ok=True)

print("Output folders:", os.listdir(OUTPUT_DIR))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

try:
    import geopandas as gpd
    import folium
except ImportError:
    print("geopandas/folium not installed. Install them if you want map visualisations.")

try:
    import plotly.express as px
except ImportError:
    print("plotly not installed. Install it if you want the dashboard export.")


Output folders: ['maps', 'histograms', 'scenarios', 'radar']


# 2. Original Code

This section contains your original RCI computation script, included verbatim.

## 2.1 Main RCI Script

In [ ]:
# -*- coding: utf-8 -*-

"""# Regenerative Capacity Index — Colab (GitHub Raw, Pandas-only)

This notebook loads your **metadata** and per-indicator **CSV time series** directly from **GitHub raw URLs** (no Google Drive mount, no DuckDB).
It applies **5th–95th percentile normalization** (or your normative caps if provided), computes **pillar scores** and a **composite index** using **equal weights** and **geometric means**.

## Repository structure (4-pillar folders)

Place this structure in your GitHub repo:

```
RCI/
├── data/
│   ├── Environmental/           # e.g., pm25.csv, no2.csv, tree_canopy.csv
│   ├── Economic/                # e.g., recycling_rate.csv, waste_per_capita.csv
│   ├── Social/                  # e.g., participation_index.csv, greenspace_300m.csv
│   └── Political/               # e.g., transparency_index.csv
├── metadata/
│   └── RCI_metadata_updated.xlsx
├── manifests/
│   └── values_manifest_github.csv   # indicator, relpath, year_col, value_col
├── outputs/                      # created by Colab
└── notebooks/
    └── RCI_Colab_from_GitHub.ipynb
```

## 1) Configure your GitHub repo and paths

- **BASE_URL** points to your repo's *raw* content (replace `<user>`, `<repo>`, `<branch>`).
- **METADATA_URL** points to `metadata/RCI_metadata_updated.xlsx` in your repo.
- **MANIFEST_URL** points to `manifests/values_manifest_github.csv`.
"""

# >>> EDIT THESE THREE LINES <<<
USER   = "<sergiofspedro>"
REPO   = "<Regenerative_potential_index>"
BRANCH = "main"  # or 'master' or another branch name

BASE_URL     = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}"
METADATA_URL = f"{BASE_URL}/tree/main/Regenerative_potential_index/metadata"
MANIFEST_URL = f"{BASE_URL}/manifests/values_manifest_github.csv"

OUTPUT_XLSX  = "RCI_outputs.xlsx"  # Notebook will save this in Colab session; download or push to GitHub manually

"""### Example: referencing a CSV within a pillar folder

If your manifest has a row like:
```
indicator,relpath,year_col,value_col
PM2.5 annual mean,data/Environmental/pm25.csv,year,value
```

Then the notebook will build the URL as:
```
https://raw.githubusercontent.com/<user>/<repo>/<branch>/data/Environmental/pm25.csv
```
and load it with `pd.read_csv(url)`.

## 2) Load metadata and manifest from GitHub

- **metadata** must include at least: `indicator`, `pillar`, `direction` ("+" or "-").
- Optional `min_cap` and `max_cap` will override empirical 5th–95th caps.
- **manifest** lists each indicator CSV **relative to the repo root** and the names of its year/value columns.
"""

import pandas as pd
import numpy as np

# Load metadata (Excel) from GitHub raw
meta = pd.read_excel(METADATA_URL, sheet_name="RCI_metadata.xlsx")
need_cols = {"indicator","pillar","direction"}
missing = need_cols - set(meta.columns)
if missing:
    raise ValueError(f"Metadata missing columns: {missing}")
meta["indicator"] = meta["indicator"].astype(str)
meta["pillar"]    = meta["pillar"].astype(str)
meta["direction"] = meta["direction"].astype(str).str.strip().replace({"plus":"+","minus":"-"})

for c in ["min_cap","max_cap"]:
    if c not in meta.columns:
        meta[c] = ""

# Load manifest (CSV) from GitHub raw
man = pd.read_csv(MANIFEST_URL)
need_cols = {"indicator","relpath","year_col","value_col"}
missing = need_cols - set(man.columns)
if missing:
    raise ValueError(f"Manifest missing columns: {missing}")
man["indicator"] = man["indicator"].astype(str)

"""## 3) Fetch indicator CSVs from GitHub and assemble values table"""

frames = []
for _, r in man.iterrows():
    ind   = r["indicator"]
    rel   = r["relpath"].lstrip("/")
    ycol  = r["year_col"]
    vcol  = r["value_col"]
    url   = f"{BASE_URL}/{rel}"
    df_i  = pd.read_csv(url)
    if ycol not in df_i.columns or vcol not in df_i.columns:
        raise ValueError(f"CSV {url} must contain columns '{ycol}' and '{vcol}'")
    sub = df_i[[ycol, vcol]].copy()
    sub.columns = ["year","value"]
    sub["indicator"] = ind
    frames.append(sub)

values = pd.concat(frames, ignore_index=True)
values["year"]  = pd.to_numeric(values["year"], errors="coerce").astype("Int64")
values["value"] = pd.to_numeric(values["value"], errors="coerce")

# Merge with metadata
d = values.merge(meta, on="indicator", how="left")
if d["direction"].isna().any():
    missing = d[d["direction"].isna()]["indicator"].unique().tolist()
    raise ValueError(f"Indicators missing from metadata: {missing}")

"""## 4) Normalize (5th–95th per indicator unless overridden by metadata caps)
- Direction: `+` higher is better, `-` lower is better.
- If an indicator has few years, 5th–95th reduces to min–max; we add a small widening if needed.
"""

def caps_from_data(series: pd.Series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if len(s) == 0:
        return np.nan, np.nan
    if len(s) < 10:
        lo, hi = s.min(), s.max()
    else:
        lo, hi = s.quantile(0.05), s.quantile(0.95)
    if not np.isfinite(lo) or not np.isfinite(hi) or lo == hi:
        hi = (lo if np.isfinite(lo) else 0.0) + (abs(lo)*0.1 + 1e-6)
    return lo, hi

rows = []
for ind, grp in d.groupby("indicator"):
    lo_meta = pd.to_numeric(grp["min_cap"], errors="coerce").dropna()
    hi_meta = pd.to_numeric(grp["max_cap"], errors="coerce").dropna()
    if len(lo_meta) and len(hi_meta):
        lo, hi = float(lo_meta.iloc[0]), float(hi_meta.iloc[0])
    else:
        lo, hi = caps_from_data(grp["value"])

    g = grp.copy()
    x = g["value"].clip(lower=lo, upper=hi)
    direction = (g["direction"].iloc[0] or "+").strip()
    if direction == "-":
        score = (hi - x) / (hi - lo) * 100.0
    else:
        score = (x - lo) / (hi - lo) * 100.0

    g["min_cap_used"] = lo
    g["max_cap_used"] = hi
    g["score_0_100"]  = score.clip(lower=0, upper=100)
    rows.append(g)

scored = pd.concat(rows, ignore_index=True) if rows else d.copy()

"""## 5) Aggregate to pillars and composite (equal weights, geometric mean)"""

def geom_mean(series: pd.Series):
    s = pd.to_numeric(series, errors="coerce").dropna().clip(lower=1e-6)/100.0
    if len(s)==0: return np.nan
    return float(np.exp(np.mean(np.log(s))) * 100.0)

pillar_rows = []
for (year, pillar), grp in scored.groupby(["year","pillar"], dropna=False):
    ps = geom_mean(grp["score_0_100"])
    pillar_rows.append({"year":year, "pillar":pillar, "pillar_score":ps})
pillars = pd.DataFrame(pillar_rows)

comp_rows = []
for year, grp in pillars.groupby("year", dropna=False):
    comp = geom_mean(grp["pillar_score"])
    comp_rows.append({"year":year, "composite_score":comp})
composite = pd.DataFrame(comp_rows)

"""## 6) Save outputs"""

with pd.ExcelWriter(OUTPUT_XLSX) as w:
    scored.to_excel(w, index=False, sheet_name="indicator_scores")
    pillars.to_excel(w, index=False, sheet_name="pillar_scores")
    composite.to_excel(w, index=False, sheet_name="composite_scores")

print("Saved:", OUTPUT_XLSX)

"""### Tips
- Keep CSV column names consistent (`year`, `value`) or adjust in the manifest `year_col`/`value_col`.
- For normative caps/targets (e.g., WHO AQG, EU noise, canopy %, cycling %, recycling %), set `min_cap`/`max_cap` in metadata.
- To add multiple cities, include a `city` column in each CSV and group by `(year, city, pillar)` similarly.
"""

HTTPError: HTTP Error 404: Not Found

# 3. Amsterdam District Processing (from manifest)

This section reads the `values_manifest_github` file, loads each indicator Excel file
with columns for year, district (`name`) and value, and builds a combined dataframe
of indicators per district and year.

The result is two dataframes:

- `district_raw`  — wide table of raw indicator values per district and year  
- `district_scores` — a simple illustrative RCI-like score per district and year  

In your final version, you can replace the dummy scoring here with your full RCI logic,
using the metadata file to map indicators to pillars and apply weights.


In [ ]:

from functools import reduce

# Try to load manifest (xlsx preferred, fallback csv)
manifest_path_xlsx = os.path.join("manifests", "values_manifest_github.xlsx")
manifest_path_csv  = os.path.join("manifests", "values_manifest_github.csv")

if os.path.exists(manifest_path_xlsx):
    manifest = pd.read_excel(manifest_path_xlsx)
elif os.path.exists(manifest_path_csv):
    manifest = pd.read_csv(manifest_path_csv)
else:
    raise FileNotFoundError("values_manifest_github.xlsx or .csv not found in manifests/")

print("Loaded manifest with", len(manifest), "indicators.")

indicator_tables = []

for _, row in manifest.iterrows():
    indicator = row["indicator"]
    relpath = row["relpath"]
    year_col = row["year_col"]
    parish_col = row["parish_col"]
    value_col = row["value_col"]

    fp = os.path.join("data", relpath)
    if not os.path.exists(fp):
        print("Warning: file not found for indicator", indicator, "->", fp)
        continue

    if fp.lower().endswith((".xlsx", ".xls")):
        df = pd.read_excel(fp)
    else:
        df = pd.read_csv(fp)

    sub = df[[year_col, parish_col, value_col]].copy()
    sub.columns = ["year", "district", indicator]
    indicator_tables.append(sub)

if not indicator_tables:
    raise RuntimeError("No indicator tables were loaded. Check manifest and data paths.")

district_raw = reduce(
    lambda left, right: pd.merge(left, right, on=["district", "year"], how="outer"),
    indicator_tables
)

print("district_raw shape:", district_raw.shape)
district_raw.to_csv(os.path.join(OUTPUT_DIR, "district_raw_indicators.csv"), index=False)

# --- Simple illustrative scoring (to be replaced by full RCI logic) ---
value_cols = [c for c in district_raw.columns if c not in ["district", "year"]]
norm = district_raw.copy()

for col in value_cols:
    col_min = norm[col].min()
    col_max = norm[col].max()
    if pd.isna(col_min) or pd.isna(col_max) or col_max == col_min:
        norm[col] = 0.0
    else:
        norm[col] = (norm[col] - col_min) / (col_max - col_min)

# For illustration, treat all indicators equally and compute a simple average
norm["RCI_overall"] = norm[value_cols].mean(axis=1)

# For now, copy overall into the four pillars (placeholder structure)
norm["RCI_ecological"] = norm["RCI_overall"]
norm["RCI_social"] = norm["RCI_overall"]
norm["RCI_economic"] = norm["RCI_overall"]
norm["RCI_political"] = norm["RCI_overall"]

district_scores = norm[["district", "year",
                        "RCI_overall",
                        "RCI_ecological",
                        "RCI_social",
                        "RCI_economic",
                        "RCI_political"]].copy()

district_scores.to_csv(os.path.join(OUTPUT_DIR, "district_scores.csv"), index=False)
print("district_scores head:")
print(district_scores.head())


# 4. Map Visualisation (Amsterdam districts)

This section creates an interactive Folium map of Amsterdam, with each district
coloured by its overall RCI score. It expects a GeoJSON file such as `geojson_lnglat.json`
that contains a polygon for each district with a property (e.g. `Stadsdeel`) matching
the `district_scores['district']` values.

In [ ]:

if 'district_scores' not in globals():
    district_scores = pd.read_csv(os.path.join(OUTPUT_DIR, "district_scores.csv"))

geojson_path = "geojson_lnglat.json"
if not os.path.exists(geojson_path):
    print("GeoJSON file not found:", geojson_path)
else:
    try:
        ams_gdf = gpd.read_file(geojson_path)
    except Exception as e:
        print("Error reading GeoJSON:", e)
        ams_gdf = None

    if ams_gdf is not None:
        # Try to match on 'Stadsdeel' property
        key_geo = "Stadsdeel" if "Stadsdeel" in ams_gdf.columns else "district"
        merged = ams_gdf.merge(district_scores, left_on=key_geo, right_on="district", how="left")

        import matplotlib.cm as cm
        import matplotlib.colors as colors

        vmin = merged["RCI_overall"].min()
        vmax = merged["RCI_overall"].max()
        cmap = cm.get_cmap("plasma")

        def value_to_color(value, vmin, vmax, cmap):
            if pd.isna(value):
                return "#cccccc"
            if vmax == vmin:
                norm = 0.5
            else:
                norm = (value - vmin) / (vmax - vmin)
            r, g, b, _ = cmap(norm)
            return colors.rgb2hex((r, g, b))

        import folium
        m = folium.Map(location=[52.37, 4.9], zoom_start=11)

        for _, row in merged.iterrows():
            score = row["RCI_overall"]
            color = value_to_color(score, vmin, vmax, cmap)
            district_name = row["district"]
            geom = row["geometry"]

            gj = folium.GeoJson(
                geom,
                style_function=lambda feature, color=color: {
                    "fillColor": color,
                    "color": "black",
                    "weight": 1,
                    "fillOpacity": 0.7,
                },
                tooltip=folium.Tooltip(
                    f"<b>District:</b> {district_name}<br><b>RCI overall:</b> {score:.3f}"
                    if pd.notna(score)
                    else f"<b>District:</b> {district_name}<br>No RCI value"
                ),
            )
            gj.add_to(m)

        out_map = os.path.join(MAPS_DIR, "RCI_Amsterdam_district_map.html")
        m.save(out_map)
        print("Saved map to:", out_map)


# 5. Histogram Generation per District

This section generates a bar chart for each district, showing the four RCI dimensions:
ecological, social, economic and political.

In [ ]:

if 'district_scores' not in globals():
    district_scores = pd.read_csv(os.path.join(OUTPUT_DIR, "district_scores.csv"))

for district, group in district_scores.groupby("district"):
    latest = group.sort_values("year").iloc[-1]
    categories = ["RCI_ecological", "RCI_social", "RCI_economic", "RCI_political"]
    values = [latest[c] for c in categories]

    plt.figure(figsize=(6, 5))
    bars = plt.bar(["Ecological", "Social", "Economic", "Political"], values,
                   color=["#2b83ba", "#abdda4", "#fdae61", "#d7191c"])
    plt.title(f"RCI by dimension – {district}")
    plt.ylabel("Score")
    ymax = max(values) if len(values) else 1
    plt.ylim(0, ymax * 1.1 if ymax > 0 else 1)

    for bar, val in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width() / 2,
                 bar.get_height(),
                 f"{val:.2f}",
                 ha="center", va="bottom", fontsize=9)

    plt.tight_layout()
    fname = os.path.join(HIST_DIR, f"RCI_dimensions_{district.replace(' ', '_')}.png")
    plt.savefig(fname, dpi=200)
    plt.close()

print("Saved histograms to:", HIST_DIR)


# 6. Radar Charts per District

This section generates radar (spider) charts for each district, using the four RCI
dimensions as axes. Charts are saved in the `RCI_outputs/radar/` folder.

In [ ]:

os.makedirs(RADAR_DIR, exist_ok=True)

if 'district_scores' not in globals():
    district_scores = pd.read_csv(os.path.join(OUTPUT_DIR, "district_scores.csv"))

def radar_chart(row):
    labels = ["Ecological", "Social", "Economic", "Political"]
    values = [row["RCI_ecological"], row["RCI_social"], row["RCI_economic"], row["RCI_political"]]
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False)
    values_c = np.concatenate((values, [values[0]]))
    angles_c = np.concatenate((angles, [angles[0]]))

    plt.figure(figsize=(6, 6))
    ax = plt.subplot(111, polar=True)
    ax.plot(angles_c, values_c, linewidth=2)
    ax.fill(angles_c, values_c, alpha=0.25)
    ax.set_xticks(angles)
    ax.set_xticklabels(labels)
    plt.title(f"RCI radar – {row['district']}")
    out = os.path.join(RADAR_DIR, f"RCI_radar_{row['district'].replace(' ', '_')}.png")
    plt.savefig(out, dpi=200)
    plt.close()

latest_scores = district_scores.sort_values("year").groupby("district").tail(1)
latest_scores.apply(radar_chart, axis=1)

print("Radar charts saved to:", RADAR_DIR)


# 7. Correlation Matrix of Indicators

This section computes a correlation matrix over all numeric indicator columns in
`district_raw` and displays a heatmap.

In [ ]:

if 'district_raw' not in globals():
    district_raw = pd.read_csv(os.path.join(OUTPUT_DIR, "district_raw_indicators.csv"))

num_cols = [c for c in district_raw.columns if c not in ["district", "year"]]
corr = district_raw[num_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation matrix of indicators (district-level)")
plt.tight_layout()
plt.show()


# 8. Scenario Engine (one-step)

This section defines a simple scenario engine that perturbs selected indicators by
percentage changes and recomputes a new `district_raw` and `district_scores` for each scenario.

In [ ]:

def run_scenario(data, scenario_dict):
    new_data = data.copy()
    for indicator, change in scenario_dict.items():
        if indicator in new_data.columns:
            new_data[indicator] = new_data[indicator] * (1 + change)
    return new_data

# Example scenarios (adapt these names and indicators to your reality)
scenarios = {
    "LOW_CARBON": {"env_indicator_toy": -0.10},
    "SOCIAL_EQUITY": {"soc_indicator_toy": 0.15},
}

if 'district_raw' not in globals():
    district_raw = pd.read_csv(os.path.join(OUTPUT_DIR, "district_raw_indicators.csv"))

for name, sc in scenarios.items():
    scenario_raw = run_scenario(district_raw, sc)
    # Recompute simple RCI scores (placeholder) for the scenario
    value_cols = [c for c in scenario_raw.columns if c not in ["district", "year"]]
    norm_s = scenario_raw.copy()
    for col in value_cols:
        col_min = norm_s[col].min()
        col_max = norm_s[col].max()
        if pd.isna(col_min) or pd.isna(col_max) or col_max == col_min:
            norm_s[col] = 0.0
        else:
            norm_s[col] = (norm_s[col] - col_min) / (col_max - col_min)
    norm_s["RCI_overall"] = norm_s[value_cols].mean(axis=1)
    norm_s["RCI_ecological"] = norm_s["RCI_overall"]
    norm_s["RCI_social"] = norm_s["RCI_overall"]
    norm_s["RCI_economic"] = norm_s["RCI_overall"]
    norm_s["RCI_political"] = norm_s["RCI_overall"]

    scenario_scores = norm_s[["district", "year",
                              "RCI_overall",
                              "RCI_ecological",
                              "RCI_social",
                              "RCI_economic",
                              "RCI_political"]].copy()
    out_csv = os.path.join(SCENARIO_DIR, f"district_scores_{name}.csv")
    scenario_scores.to_csv(out_csv, index=False)
    print("Saved scenario scores:", out_csv)


# 9. Time-Series Scenario Engine

This section creates a simple time-series scenario by applying the same percentage changes
across future time steps (e.g., 2030, 2040, 2050) starting from a base year.

In [ ]:

def run_time_series_scenario(df, scenario_changes, years=[2020, 2030, 2040, 2050]):
    # Assume df contains a single base year already
    base_year = df['year'].min()
    base = df[df['year'] == base_year].copy()
    frames = []
    for i, year in enumerate(years):
        temp = base.copy()
        for indicator, change in scenario_changes.items():
            if indicator in temp.columns:
                factor = (1 + change) ** i
                temp[indicator] = temp[indicator] * factor
        temp["year"] = year
        frames.append(temp)
    return pd.concat(frames, ignore_index=True)

# Example story for time-series (use one of the indicators present)
time_scenario = {"env_indicator_toy": 0.03}

ts_raw = run_time_series_scenario(district_raw, time_scenario)
ts_raw.to_csv(os.path.join(SCENARIO_DIR, "district_raw_time_series.csv"), index=False)
print("Saved time-series raw indicators for scenario to:", os.path.join(SCENARIO_DIR, "district_raw_time_series.csv"))


# 10. Story-Based Scenarios

This section defines narrative scenarios (e.g., 'Green Transition', 'Climate Shock') and
maps them to indicator changes using the `run_scenario` function.

In [ ]:

stories = {
    "Green_Transition": {"env_indicator_toy": 0.10},
    "Climate_Shock": {"eco_indicator_toy": -0.05},
    "Governance_Reform": {"pol_indicator_toy": 0.15},
}

for story, changes in stories.items():
    out_raw = run_scenario(district_raw, changes)
    out_path = os.path.join(SCENARIO_DIR, f"district_raw_story_{story}.csv")
    out_raw.to_csv(out_path, index=False)
    print("Saved story-based raw indicators:", out_path)


# 11. Plotly Dashboard Export (static for GitHub Pages)

This section builds a static HTML dashboard in `docs/index.html` using Plotly,
so that it can be served by GitHub Pages.

In [ ]:

os.makedirs("docs", exist_ok=True)

if 'district_scores' not in globals():
    district_scores = pd.read_csv(os.path.join(OUTPUT_DIR, "district_scores.csv"))

# Use latest year per district
latest_scores = district_scores.sort_values("year").groupby("district").tail(1)

try:
    import plotly.express as px
except ImportError:
    raise ImportError("plotly is required for this section. Install it with `pip install plotly`.")

fig_bar = px.bar(
    latest_scores,
    x="district",
    y="RCI_overall",
    title="Overall Regenerative Capacity Index by Amsterdam District",
    labels={"district": "District", "RCI_overall": "RCI overall score"},
)

fig_scatter = px.scatter(
    latest_scores,
    x="RCI_ecological",
    y="RCI_social",
    color="RCI_overall",
    hover_name="district",
    title="Ecological vs Social RCI scores by district",
    labels={"RCI_ecological": "Ecological score", "RCI_social": "Social score"},
    color_continuous_scale="Viridis",
)

html_parts = []
html_parts.append("<html><head><title>RCI Amsterdam Dashboard</title></head><body>")
html_parts.append("<h1>Regenerative Capacity Index – Amsterdam Dashboard</h1>")

html_parts.append("<h2>Overall RCI by District</h2>")
html_parts.append(fig_bar.to_html(full_html=False, include_plotlyjs="cdn"))

html_parts.append("<h2>Ecological vs Social scores</h2>")
html_parts.append(fig_scatter.to_html(full_html=False, include_plotlyjs=False))

html_parts.append("</body></html>")

with open(os.path.join("docs", "index.html"), "w", encoding="utf-8") as f:
    f.write("\n".join(html_parts))

print("Static Plotly dashboard written to docs/index.html")


# 12. GitHub Actions Builder (workflow + build script)

This section writes a minimal `build_plotly_dashboard.py` script and a GitHub Actions
workflow file `deploy.yml` to `.github/workflows/`, which can be used to automatically
build and deploy the dashboard to GitHub Pages.

In [ ]:
os.makedirs(".github/workflows", exist_ok=True)

build_script = '''
import pandas as pd
import plotly.express as px
import os

os.makedirs("docs", exist_ok=True)
scores = pd.read_csv("RCI_outputs/district_scores.csv")
latest_scores = scores.sort_values("year").groupby("district").tail(1)

fig_bar = px.bar(
    latest_scores,
    x="district",
    y="RCI_overall",
    title="Overall Regenerative Capacity Index by Amsterdam District",
    labels={"district": "District", "RCI_overall": "RCI overall score"},
)

fig_scatter = px.scatter(
    latest_scores,
    x="RCI_ecological",
    y="RCI_social",
    color="RCI_overall",
    hover_name="district",
    title="Ecological vs Social RCI scores by district",
    labels={"RCI_ecological": "Ecological score", "RCI_social": "Social score"},
    color_continuous_scale="Viridis",
)

html_parts = []
html_parts.append("<html><head><title>RCI Amsterdam Dashboard</title></head><body>")
html_parts.append("<h1>Regenerative Capacity Index – Amsterdam Dashboard</h1>")
html_parts.append("<h2>Overall RCI by District</h2>")
html_parts.append(fig_bar.to_html(full_html=False, include_plotlyjs="cdn"))
html_parts.append("<h2>Ecological vs Social scores</h2>")
html_parts.append(fig_scatter.to_html(full_html=False, include_plotlyjs=False))
html_parts.append("</body></html>")

with open(os.path.join("docs", "index.html"), "w", encoding="utf-8") as f:
    f.write("\n".join(html_parts))

print("Dashboard built at docs/index.html")
'''

with open("build_plotly_dashboard.py", "w", encoding="utf-8") as f:
    f.write(build_script)

workflow = '''
name: Build and Deploy RCI Dashboard

on:
  push:
    branches: [ main ]
  workflow_dispatch:

permissions:
  contents: read
  pages: write
  id-token: write

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout repo
        uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.11"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install pandas plotly

      - name: Build dashboard
        run: |
          python build_plotly_dashboard.py

      - name: Upload artifact
        uses: actions/upload-pages-artifact@v3
        with:
          path: docs

  deploy:
    runs-on: ubuntu-latest
    needs: build
    environment:
      name: github-pages
      url: ${{ steps.deployment.outputs.page_url }}
    steps:
      - name: Deploy to GitHub Pages
        id: deployment
        uses: actions/deploy-pages@v4
'''

with open(".github/workflows/deploy.yml", "w", encoding="utf-8") as f:
    f.write(workflow)

print("Created build_plotly_dashboard.py and .github/workflows/deploy.yml")
